In [ ]:
#weighet mean
#reflective boundaries
#beta =1.8
import math
import numpy as np
import matplotlib.pyplot as plt
from opfunu.cec_based import cec2014, cec2017
import matplotlib
from matplotlib.backend_bases import MouseButton
from scipy.special import gamma
from math import sin, pi
matplotlib.use('TkAgg')

def aquila_optimizer_weighted_mean(fitness_function, bounds, dim, max_evals):
    """
    Aquila Optimizer with a weighted mean factor.

    Args:
        fitness_function: The function to be minimized.
        bounds (list or tuple): A tuple or list containing the lower and upper bounds [lb, ub].
        dim (int): The dimensionality of the search space.
        max_evals (int): The maximum number of function evaluations.

    Returns:
        tuple: A tuple containing the best position found, the best fitness value,
               and the convergence curve.
    """
    population_size = 50
    max_iterations = max_evals // population_size
    lb, ub = bounds
    alpha = 0.1
    delta = 0.1
    beta = 1.8
    positions = np.random.uniform(lb, ub, (population_size, dim))
    X_best = positions[0].copy()
    best_fitness = float('inf')
    convergence_curve = []

    for i in range(population_size):
        current_fitness = fitness_function(positions[i])
        if current_fitness < best_fitness:
            best_fitness = current_fitness
            X_best = positions[i].copy()

    for iter in range(max_iterations):
        fitness_values = np.array([fitness_function(pos) for pos in positions])
        weights = 1 / (fitness_values + np.finfo(float).eps)  # Avoid division by zero
        weighted_mean = np.sum(positions * weights[:, np.newaxis], axis=0) / np.sum(weights)

        for i in range(population_size):
            if iter <= (2/3)*max_iterations:
                if np.random.rand() < 0.5:
                    positions[i] = X_best * (1 - iter/max_iterations) + (weighted_mean - X_best) * np.random.rand()
                else:
                    r = np.random.rand()
                    theta = np.random.rand() * 2 * np.pi
                    x = r * np.sin(theta)
                    y = r * np.cos(theta)
                    sigma = (gamma(1 + beta)* sin(pi * beta / 2)/ (gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
                    u = np.random.randn(dim) * sigma
                    v = np.random.randn(dim)
                    levy = 0.01 * u / np.abs(v)**(1/beta)
                    positions[i] = X_best * levy + positions[np.random.randint(population_size)] + (y - x) * np.random.rand()
            else:
                if np.random.rand() < 0.5:
                    positions[i] = (X_best * weighted_mean) * alpha - np.random.rand() + ((ub - lb) * np.random.rand() + lb) * delta
                else:
                    QF = iter**((2*np.random.rand()-1)/(1 - max_iterations)**2)
                    G1 = 2 * np.random.rand() - 1
                    G2 = 2 * (1 - iter/max_iterations)
                    if dim > 1:
                        levy_vec = np.random.rand(dim)
                        rand_vec = np.random.rand(dim)
                        positions[i] = QF * X_best - G1 * positions[i] * np.random.rand() - G2 * levy_vec + rand_vec * G1
                    else:
                        levy = np.random.rand()
                        positions[i] = QF * X_best - G1 * positions[i] * np.random.rand() - G2 * levy + np.random.rand() * G1

            def reflective_boundaries(position, lb, ub):
                position = np.where(position < lb, 2 * lb - position, position)
                position = np.where(position > ub, 2 * ub - position, position)
                return position

            positions[i] = reflective_boundaries(positions[i], lb, ub)
            current_fitness = fitness_function(positions[i])
            if current_fitness < best_fitness:
                best_fitness = current_fitness
                X_best = positions[i].copy()

        convergence_curve.append(best_fitness)

    return X_best, best_fitness, convergence_curve

def run_experiments_weighted_mean():
    cec_functions = {
        'CEC2014': [cec2014.F12014(ndim=10)],
        'CEC2017': [cec2017.F52017(ndim=10)]
        # 'CEC2020': [cec2020.F12020(ndim=10)],
        # 'CEC2022': [cec2022.F12022(ndim=10)]
    }

    convergence_data_weighted_mean = {}

    for suite, functions in cec_functions.items():
        for func in functions:
            print(f"Running AO with Weighted Mean on {suite} - {func.__class__.__name__}")
            _, _, convergence_curve = aquila_optimizer_weighted_mean(
                func.evaluate,
                [func.lb, func.ub],
                func.ndim,
                max_evals=60000
            )
            convergence_data_weighted_mean[suite] = convergence_curve

    return convergence_data_weighted_mean

convergence_data_weighted_mean = run_experiments_weighted_mean()

plt.figure(figsize=(12, 8))
for suite, data in convergence_data_weighted_mean.items():
    plt.plot(data, label=f"AO with Weighted Mean - {suite} - Function 1")

plt.xlabel("Iterations")
plt.ylabel("Fitness Value")
plt.title("Convergence Curve of Aquila Optimizer with Weighted Mean")
plt.legend()
plt.grid(True)
plt.show()